# Load Ntuple and Print MCDecayString

In [8]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import uproot
from tqdm.auto import tqdm
import sys
sys.path.append('/home/belle/zhangboy/inclusive_R_D/')
import utilities as util

training_variables = util.training_variables
columns = util.all_relevant_variables + ['__MCDecayString__']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# 4S Data vs MC

cut = '(5<B0_roeMbc_my_mask) & (-4<B0_roeDeltae_my_mask) & (B0_roeDeltae_my_mask<1)'

# Load data files
MC_4S = uproot.concatenate([f'../Samples/MC15ri/e_channel/MC15ri_1ab_quaxo_e_2/*.root:B0'],
                          library="np",
                          cut = cut,
                          filter_branch=lambda branch: branch.name in columns)

data_4S = uproot.concatenate([f'../Samples/Data/e_channel/proc13_4S_quaxo_4.root:B0'],
                          library="np",
                          cut = cut,
                          filter_branch=lambda branch: branch.name in columns)

MC_sig = uproot.concatenate([f'../Samples/MC15ri/e_channel/MC15ri_sig_quaxo_e_1/*.root:B0'],
                          library="np",
                          cut = cut,
                          filter_branch=lambda branch: branch.name in columns)

df_mc_4S = pd.DataFrame(MC_4S).sample(frac=1).reset_index(drop=True)
df_data_4S = pd.DataFrame(data_4S)
df_mc_sig = pd.DataFrame(MC_sig)

In [10]:
lgb_tight = 'sig_prob==largest_prob and sig_prob>0.65 and fakeD_prob<0.05'
MVA_all = 'D_M>0'

for df in [df_mc_4S, df_mc_sig, df_data_4S]:
    # df.eval(f'cos_D_l = (D_px*ell_px + D_py*ell_py + D_pz*ell_pz)/(D_p*ell_p)', inplace=True)
    df.eval('B_D_ReChi2 = B0_vtxReChi2 + D_vtxReChi2', inplace=True)
    df.eval('p_D_l = D_CMS_p + ell_CMS_p', inplace=True)

df_mc_4S_lgb = util.apply_mva_bcs(df_mc_4S, training_variables, lgb_tight, library='lgbm')
df_mc_sig_lgb = util.apply_mva_bcs(df_mc_sig, training_variables, lgb_tight, library='lgbm')
df_data_4S_lgb = util.apply_mva_bcs(df_data_4S, training_variables, lgb_tight, library='lgbm')

In [23]:
samples_generic = util.classify_mc_dict(df_mc_4S_lgb, 'e', template=False)
samples_sig = util.classify_mc_dict(df_mc_sig_lgb, 'e', template=False)
mpl=util.mpl(samples_generic)
for name, df in samples_generic.items():
    print(name, len(df))

bkg_fakeD 47226
bkg_TDFl 6410
bkg_fakeTracks 2504
bkg_continuum 13203
bkg_combinatorial 15124
bkg_singleBbkg 9611
bkg_other_TDTl 0
$D\tau\nu$ 4241
$D^\ast\tau\nu$ 2483
$D\ell\nu$ 90282
$D^\ast\ell\nu$ 57810
$D^{\ast\ast}\tau\nu$ 1472
$D^{\ast\ast}\ell\nu$_narrow 18373
$D^{\ast\ast}\ell\nu$_broad 14110
$D\ell\nu$_gap_pi 3467
$D\ell\nu$_gap_eta 3076
bkg_other_signal 0


In [25]:
samples_generic = util.classify_mc_dict(df_mc_4S_lgb, 'e', template=False)
samples_sig = util.classify_mc_dict(df_mc_sig_lgb, 'e', template=False)
mpl=util.mpl(samples_generic)
for name, df in samples_generic.items():
    print(name, len(df))

bkg_fakeD 47226
bkg_TDFl 6410
bkg_fakeTracks 2504
bkg_continuum 13203
bkg_combinatorial 15124
bkg_singleBbkg 9611
bkg_other_TDTl 0
$D\tau\nu$ 4241
$D^\ast\tau\nu$ 2483
$D\ell\nu$ 90282
$D^\ast\ell\nu$ 57810
$D^{\ast\ast}\tau\nu$ 1472
$D^{\ast\ast}\ell\nu$_narrow 18373
$D^{\ast\ast}\ell\nu$_broad 14110
$D\ell\nu$_gap_pi 3467
$D\ell\nu$_gap_eta 3076
bkg_other_signal 0


In [22]:
samples_generic['bkg_fakeTracks'].query('D_mcErrors==512')[['ell_mcErrors','ell_mcPDG','ell_genMotherPDG','ell_genGMPDG',
         'D_mcErrors','D_mcPDG','D_genMotherPDG','B0_isContinuumEvent'
]]

,ell_mcErrors,ell_mcPDG,ell_genMotherPDG,ell_genGMPDG,D_mcErrors,D_mcPDG,D_genMotherPDG,B0_isContinuumEvent
2650666,512.0,NaN,0.0,0.0,0.0,-411.0,-10421.0,False
5450912,512.0,NaN,0.0,0.0,304.0,23.0,0.0,True
286073,512.0,NaN,0.0,0.0,0.0,-411.0,30443.0,False
8157105,512.0,NaN,0.0,0.0,0.0,411.0,413.0,False
8979554,512.0,NaN,0.0,0.0,376.0,300553.0,0.0,False
...,...,...,...,...,...,...,...,...
1933654,512.0,NaN,0.0,0.0,0.0,411.0,23.0,True
3238042,512.0,NaN,0.0,0.0,368.0,23.0,0.0,True
9932940,512.0,NaN,0.0,0.0,432.0,23.0,0.0,True
10393256,512.0,NaN,0.0,0.0,0.0,-411.0,23.0,True


In [17]:
df_test = df_mc_4S_lgb.query('B0_mcErrors==512 and D_mcErrors!=512 and ell_mcErrors!=512')
df_test[['ell_mcErrors','ell_mcPDG','ell_genMotherPDG','ell_genGMPDG',
         'D_mcErrors','D_mcPDG','D_genMotherPDG','B0_isContinuumEvent'
]]

,ell_mcErrors,ell_mcPDG,ell_genMotherPDG,ell_genGMPDG,D_mcErrors,D_mcPDG,D_genMotherPDG,B0_isContinuumEvent
10120113,0.0,-11.0,22.0,11.0,0.0,-411.0,23.0,True
7321169,0.0,-11.0,22.0,11.0,368.0,23.0,0.0,True
7961508,0.0,-11.0,22.0,11.0,352.0,23.0,0.0,True
8878773,0.0,-11.0,22.0,11.0,432.0,23.0,0.0,True
8273894,0.0,-11.0,22.0,11.0,432.0,23.0,0.0,True
1704054,2048.0,11.0,22.0,11.0,0.0,411.0,413.0,True
8174128,0.0,-11.0,22.0,11.0,496.0,23.0,0.0,True
8090351,0.0,-11.0,22.0,11.0,0.0,-411.0,-415.0,True
2285363,0.0,-11.0,22.0,11.0,436.0,23.0,0.0,True
6240005,2048.0,11.0,22.0,11.0,0.0,411.0,413.0,True


In [5]:
df_test['B0_mcPDG'] = df_test['B0_mcPDG'].fillna(0)
df_test['D_mcPDG'] = df_test['D_mcPDG'].fillna(0)

df_test.eval(f'p_D_l = D_CMS_p + e_CMS_p', inplace=True)

index_list = [412, 1016, 1101, 1109, 1263, 1751]
df_test4 = df_test.query('index in @index_list')
df_test4[['B0_B0Mode','B0_Bbar0Mode','B0_mcErrors','B0_mcPDG','D_mcErrors','D_mcPDG','D_genMotherPDG',
          'e_genMotherPDG','e_genGMPDG','e_mcErrors','e_mcPDG','B0_mcDaughter_0_PDG',
          'B0_mcDaughter_1_PDG']]

In [ ]:
#df2=df_test.query('B0_mcErrors>=1176 and abs(B0_mcPDG)!=300553 and D_mcErrors==0 and e_mcErrors>=128 and e_genMotherPDG==B0_mcPDG and abs(e_mcPDG)==11')
candidate = df_test.iloc[0]
print(candidate['__MCDecayString__'])

# e channel

In [16]:
df3 = samples[r'$D\ell\nu$'].query('B0_CMS3_weMissM2>8 and p_D_l<1')
df3[['__event__','B0_mcErrors','e_mcErrors','B0_CMS_p','D_CMS_mcP','D_CMS_p','e_CMS_mcP','e_CMS_p','e_pSig','B0_CMS3_weMissM2','p_D_l']]

,__event__,B0_mcErrors,e_mcErrors,B0_CMS_p,D_CMS_mcP,D_CMS_p,e_CMS_mcP,e_CMS_p,e_pSig,B0_CMS3_weMissM2,p_D_l
38740,8198493,2056.0,2048.0,0.961089,1.328134,1.320325,1.583429,1.026613,56.063556,2.977534,2.346938
39404,1372477,2188.0,2180.0,1.450823,1.031608,1.032232,1.240519,0.826538,59.536626,3.322220,1.858770
55971,34434797,8.0,0.0,1.338455,1.148966,1.149381,1.154281,0.509834,362.513188,4.905236,1.659215
73278,5939869,8.0,0.0,0.993786,1.051094,1.050751,1.488461,0.708447,591.061317,4.959367,1.759197
119911,2459735,8.0,0.0,1.159548,1.288300,1.290189,1.322123,0.815593,190.065856,3.518582,2.105782
126545,8748384,2056.0,2048.0,1.654234,1.317880,1.281719,1.232314,0.738880,44.955691,2.584715,2.020599
179925,22575596,8.0,0.0,1.008443,1.528610,1.526374,1.937666,0.628086,877.360666,4.025019,2.154461
247727,37154896,8.0,0.0,1.140189,1.113648,1.112609,1.437532,1.100309,168.007385,2.758081,2.212918
251989,9089867,8.0,0.0,1.286374,1.656673,1.655751,1.908782,0.506599,481.262032,3.566608,2.162351
261857,18733482,8.0,0.0,1.477670,1.225286,1.222806,1.003199,0.521974,201.632287,4.241776,1.744780


In [14]:
pd.set_option('display.max_rows', None)
b=df.query('B0_mcErrors==24')[['B0_mcPDG','D_mcPDG','e_mcPDG','e_genMotherPDG','D_mcErrors','B0_mcErrors','e_mcErrors','DecayMode']].value_counts()
b

B0_mcPDG  D_mcPDG  e_mcPDG  e_genMotherPDG  D_mcErrors  B0_mcErrors  e_mcErrors  DecayMode
 511.0    -411.0   -11.0     511.0          0.0         24.0         0.0         4            15939
-511.0     411.0    11.0    -511.0          0.0         24.0         0.0         4            15621
 511.0    -411.0   -11.0     511.0          0.0         24.0         0.0         6             5844
-511.0     411.0    11.0    -511.0          0.0         24.0         0.0         6             5752
 511.0    -411.0   -11.0     431.0          0.0         24.0         0.0         0               40
-511.0     411.0    11.0     15.0           0.0         24.0         0.0         3               34
 511.0    -411.0   -11.0    -15.0           0.0         24.0         0.0         3               23
-511.0     411.0    11.0    -431.0          0.0         24.0         0.0         0               21
                             15.0           0.0         24.0         0.0         0               18
 511.0   

# DecayHash

In [7]:
import decayHash
from decayHash import DecayHashMap
decayhash=f'../Samples/Signal_MC14ri/MC14ri_normDDst_foldex_e_7/hashmap_normDDst.root'
hashmap1 = DecayHashMap(decayhash, removeRadiativeGammaFlag=True)

In [ ]:
hash_values = candidate[['B0_DecayHash', "B0_DecayHashEx"]].values
org1 = hashmap1.get_original_decay(*hash_values)
print(org1.to_string())

In [52]:
import ROOT
mode = '-521 (-> 413 -211 111 11 -12)'
decaytree = ROOT.Belle2.DecayTree(mode)
hashmap1.get_original_decay(candidate["B0_DecayHash"],candidate["B0_DecayHashEx"]).find_decay(decaytree)

True

In [21]:
def found(modes,row):
    for mode in modes:
        # check the decay chain for the reconstructed B meson only
        if mode.startswith(str(int(candidate['B0_mcPDG']))):
            print(mode)
            decaytree = ROOT.Belle2.DecayTree(mode)
            if hashmap1.get_original_decay(candidate["B0_DecayHash"],candidate["B0_DecayHashEx"]).find_decay(decaytree):
                return True
            else:
                continue
        else:
            continue
    return False

In [ ]:
hash_modes = util.mode_dict['e']
for name,modes in hash_modes.items():
    if found(modes,candidate):
        print(util.DecayMode[name].value)
    else:
        print('No'+name)